In [1]:
import duckdb
import pandas as pd
import config

data = duckdb.connect(f"../{config.DATA_DIR_NAME}/data_tables.db")

In [2]:
data.sql("""
    select *
    from sen_age_sex
    limit 5
    """)

┌─────────────┬─────────────────┬──────────────────┬──────────────┬──────────────┬─────────────┬─────────────┬─────────────┬─────────┬─────────────┬───────────────────────────────┬──────────────────┬────────────────────────────┬─────────────────┬─────────────────┬─────────────┬───────────────┐
│ time_period │ time_identifier │ geographic_level │ country_code │ country_name │ region_name │ region_code │ old_la_code │ la_name │ new_la_code │      phase_type_grouping      │ pupil_sen_status │        primary_need        │ breakdown_topic │    breakdown    │ pupil_count │ pupil_percent │
│    int64    │     varchar     │     varchar      │   varchar    │   varchar    │   varchar   │   varchar   │    int64    │ varchar │   varchar   │            varchar            │     varchar      │          varchar           │     varchar     │     varchar     │    int64    │    double     │
├─────────────┼─────────────────┼──────────────────┼──────────────┼──────────────┼─────────────┼─────────────┼─────

In [3]:
ages = data.sql(
    """
    select time_period, phase_type_grouping, breakdown, sum(pupil_count) as pupil_count
    from sen_age_sex
    where phase_type_grouping = 'State-funded primary' and pupil_sen_status = 'EHC plans' and breakdown_topic = 'Age'
    group by time_period, pupil_sen_status, phase_type_grouping, breakdown
    order by breakdown, time_period    
    """).df()

ages.head()

,time_period,phase_type_grouping,breakdown,pupil_count
0,201516,State-funded primary,Age 10,64980.0
1,201617,State-funded primary,Age 10,64356.0
2,201718,State-funded primary,Age 10,69480.0
3,201819,State-funded primary,Age 10,79254.0
4,201920,State-funded primary,Age 10,89454.0
